In [1]:
from PIL import Image
import os
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import clear_output

In [8]:
def read_labels(path):
    img_labels = pd.read_csv(path, index_col=0, dtype=str)
    return img_labels

def save_labels(df, path):
    df.to_csv(path)
    return None

def labelingN(N, LABELS, PATH_IMG, PATH_LABEL):
    # create a label file
    if os.path.isfile(PATH_LABEL):
        img_labels = read_labels(PATH_LABEL)
    else:
        img_labels = pd.DataFrame(columns=["img"]+LABELS).set_index("img")
    # labeling
    update_count = 0
    file_imgs = os.listdir(PATH_IMG)
    np.random.shuffle(file_imgs)
    n = min(int(N), len(file_imgs))
    for file_img in file_imgs[:n]:
        if ".jpg" in file_img or ".jpeg" in file_img:
            print("labels: %a  |  img_name: %s" % (LABELS, file_img))
            img = Image.open(PATH_IMG+file_img)
            plt.figure(figsize=(12, 12))
            plt.imshow(np.array(img))
            plt.show()
            labels = [input("labeling: %s" % l) for l in LABELS]
            label_series = pd.Series(labels, index=img_labels.columns, dtype=str, name=file_img)
            if file_img in img_labels.index:
                update_count += 1
                if not all(img_labels.loc[file_img] == label_series):
                    print("Received Conflicting Labels:")
                    print(" original  : %a" % list(img_labels.loc[file_img]))
                    print("newly-added: %a" % list(label_series))
                    labels = [input("final labeling: %s" % l) for l in LABELS]
                    label_series = pd.Series(labels, index=img_labels.columns, dtype=str, name=file_img)
            img_labels = img_labels.append(label_series)
            clear_output()
    # save labels
    save_labels(img_labels, PATH_LABEL)
    return update_count

def labeling(LABELS, PATH_IMG, PATH_LABEL):
    N = -1
    all_count = 0
    update_count = 0
    while N != str(0):
        N = input("How many images do you want to label: ")
        update = labelingN(N, LABELS, PATH_IMG, PATH_LABEL)
        update_count += update
        all_count += int(N)
    img_labels = read_labels(PATH_LABEL)
    len_labels = len(img_labels)
    print("update/add: %d/%d" % (update_count, all_count - update_count))
    print("done! [%.2f%% labeled]" % (100*len_labels/len(os.listdir(PATH_IMG))))
    return None

In [54]:
### Corrosion Labeling
PATH_IMG = "../../data/images/"
PATH_LABEL = "../../data/labeled_corrosion.csv"
LABELS = ["attached", "binary", "five"]
labeling(LABELS=LABELS, PATH_IMG=PATH_IMG, PATH_LABEL=PATH_LABEL)

How many images do you want to label:  0


update/add: 0/10
done! [1.78% labeled]


In [64]:
from collections import Counter, OrderedDict
temp = read_labels(PATH_LABEL)
c = Counter(temp.five.values)
total = 0
for i in sorted(c.keys()):
    print("%s: %d" % (i, c[i]))
    total += c[i]
print("sum: %d" % total)
print("with attachement: %d" % (temp.attached == "1").sum())

1: 15
2: 16
3: 34
4: 16
5: 29
sum: 110
with attachement: 100
